In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'/home/ubuntu/varios/skforecast'

In [2]:
from sklearn.preprocessing import OneHotEncoder

In [3]:
import sklearn
sklearn.__version__

'1.4.1.post1'

In [4]:
# create an example dataframe to show the use of the OneHotEncoder
import pandas as pd
import numpy as np

df = pd.DataFrame({'_level_skforecast': ['a', 'b', 'c'], 'B': ['b', 'a', 'c'], 'C': [1, 2, 3]})

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

encoder = OneHotEncoder(categories='auto', sparse_output=False, drop=None, dtype=int).set_output(transform='pandas')
#encoder = OrdinalEncoder(categories='auto', dtype=int).set_output(transform='pandas')
self = forecaster

transformer = ColumnTransformer(
    transformers=[
        (self.encoding, encoder, ['_level_skforecast'])
    ], remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform='pandas')

X_train = transformer.fit_transform(df)
X_train

NameError: name 'forecaster' is not defined

In [ ]:
encoder.fit_transform(df[['_level_skforecast']])

,_level_skforecast_a,_level_skforecast_b,_level_skforecast_c
0,1,0,0
1,0,1,0
2,0,0,1


In [ ]:
mapping = {}
for i, code in enumerate(transformer.named_transformers_[self.encoding].categories_[0]):
     mapping[code] = i

mapping

NameError: name 'transformer' is not defined

In [ ]:
transformer.named_transformers_[self.encoding].categories_[0]

array(['a', 'b', 'c'], dtype=object)

In [6]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries

# Data download
# ==============================================================================
url = (
       'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/'
       'data/simulated_items_sales.csv'
)
data = pd.read_csv(url, sep=',')

# Data preparation
# ==============================================================================
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data = data.set_index('date')
data = data.asfreq('D')
data = data.sort_index()
data.head()

# Split data into train-val-test
# ==============================================================================
end_train = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()

print(
    f"Train dates : {data_train.index.min()} --- {data_train.index.max()}   "
    f"(n={len(data_train)})"
)
print(
    f"Test dates  : {data_test.index.min()} --- {data_test.index.max()}   "
    f"(n={len(data_test)})"
)

Train dates : 2012-01-01 00:00:00 --- 2014-07-15 00:00:00   (n=927)
Test dates  : 2014-07-16 00:00:00 --- 2015-01-01 00:00:00   (n=170)


In [130]:
# Create and fit forecaster multi series
# ==============================================================================
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123),
                 lags               = 24,
                 encoding           = 'aaaa',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 forecaster_id      = None,
                 #fit_kwargs={'categorical_feature':'auto'}
             )

forecaster.fit(series=data_train)
forecaster.encoding_mapping

lag_1                 float64
lag_2                 float64
lag_3                 float64
lag_4                 float64
lag_5                 float64
lag_6                 float64
lag_7                 float64
lag_8                 float64
lag_9                 float64
lag_10                float64
lag_11                float64
lag_12                float64
lag_13                float64
lag_14                float64
lag_15                float64
lag_16                float64
lag_17                float64
lag_18                float64
lag_19                float64
lag_20                float64
lag_21                float64
lag_22                float64
lag_23                float64
lag_24                float64
_level_skforecast    category
dtype: object
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000559 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6124
[LightGBM] [Info] Number of data points 

{'item_1': 0, 'item_2': 1, 'item_3': 2}

In [131]:
# Show categorical features if present
cat_index = forecaster.regressor.booster_.params.get('categorical_column')
if cat_index is not None:
    features = forecaster.regressor.booster_.feature_name()
    print([features[i] for i in cat_index])

['_level_skforecast']


In [132]:
Que las opciones de encoding sean: 'onehot', 'ordinal', 'ordinal_category'
Que en las predicciones se invierta el nombre del level utilizando el mapping.

SyntaxError: invalid syntax (1311491502.py, line 1)

In [134]:
forecaster.predict(steps=10, levels=['item_2'])

1
[[10.48958333 10.89583333  8.1        11.00833333 11.00208333  9.05208333
  12.67083333 11.57916667 11.1875     12.11666667 11.0625      9.6375
  12.13333333 12.39791667 10.77291667  8.40208333 10.68958333 12.26041667
  13.17708333 13.37708333 11.61666667  9.82916667 13.88333333 11.89375
   1.        ]]
1
[[10.58985244 10.48958333 10.89583333  8.1        11.00833333 11.00208333
   9.05208333 12.67083333 11.57916667 11.1875     12.11666667 11.0625
   9.6375     12.13333333 12.39791667 10.77291667  8.40208333 10.68958333
  12.26041667 13.17708333 13.37708333 11.61666667  9.82916667 13.88333333
   1.        ]]
1
[[11.24964181 10.58985244 10.48958333 10.89583333  8.1        11.00833333
  11.00208333  9.05208333 12.67083333 11.57916667 11.1875     12.11666667
  11.0625      9.6375     12.13333333 12.39791667 10.77291667  8.40208333
  10.68958333 12.26041667 13.17708333 13.37708333 11.61666667  9.82916667
   1.        ]]
1
[[11.21494489 11.24964181 10.58985244 10.48958333 10.89583333  8.1


,item_2
2014-07-16,10.589852
2014-07-17,11.249642
2014-07-18,11.214945
2014-07-19,11.392770
2014-07-20,11.092514
2014-07-21,11.367802
2014-07-22,10.610384
2014-07-23,12.443352
2014-07-24,11.563970
2014-07-25,11.425697


In [129]:
forecaster.predict(steps=10, levels=['item_2'])

1
[[10.48958333 10.89583333  8.1        11.00833333 11.00208333  9.05208333
  12.67083333 11.57916667 11.1875     12.11666667 11.0625      9.6375
  12.13333333 12.39791667 10.77291667  8.40208333 10.68958333 12.26041667
  13.17708333 13.37708333 11.61666667  9.82916667 13.88333333 11.89375
   1.        ]]
1
[[10.58985244 10.48958333 10.89583333  8.1        11.00833333 11.00208333
   9.05208333 12.67083333 11.57916667 11.1875     12.11666667 11.0625
   9.6375     12.13333333 12.39791667 10.77291667  8.40208333 10.68958333
  12.26041667 13.17708333 13.37708333 11.61666667  9.82916667 13.88333333
   1.        ]]
1
[[11.24964181 10.58985244 10.48958333 10.89583333  8.1        11.00833333
  11.00208333  9.05208333 12.67083333 11.57916667 11.1875     12.11666667
  11.0625      9.6375     12.13333333 12.39791667 10.77291667  8.40208333
  10.68958333 12.26041667 13.17708333 13.37708333 11.61666667  9.82916667
   1.        ]]
1
[[11.21494489 11.24964181 10.58985244 10.48958333 10.89583333  8.1


,item_2
2014-07-16,10.589852
2014-07-17,11.249642
2014-07-18,11.214945
2014-07-19,11.392770
2014-07-20,11.092514
2014-07-21,11.367802
2014-07-22,10.610384
2014-07-23,12.443352
2014-07-24,11.563970
2014-07-25,11.425697
